## Group Code

Image Processing

In [2]:
# @title Dependencies
import os, sys

import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
%matplotlib inline

import cv2

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

from IPython.display import display, Javascript, Image

from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import html
import time

# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # get OpenCV format image
  img = js_to_image(data)
  # save image
  cv2.imwrite(filename, img)

  return filename

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def re_encode_video(video_path):
    print(f"Re-encoding video, this might take some time, please be patient.")
    #added -r 30, otherwise, for some reason it records 1000FPS as metadata for the Astra
    os.system(f"ffmpeg -y -i {video_path} -vcodec libx264 -r 30 temp.mp4")
    os.system(f"rm {video_path}")
    os.system(f"mv temp.mp4 {video_path}")
    print(f"Done encoding!")

def record_video(filename='video.mp4'):
  js = Javascript("""
    async function recordVideo() {
      // mashes together the advanced_outputs.ipynb function provided by Colab,
      // a bunch of stuff from Stack overflow, and some sample code from:
      // https://developer.mozilla.org/en-US/docs/Web/API/MediaStream_Recording_API

      // Optional frames per second argument.
      const options = { mimeType: "video/webm; codecs=vp9" };
      const div = document.createElement('div');
      const capture = document.createElement('button');
      const stopCapture = document.createElement("button");
      capture.textContent = "Start Recording";
      capture.style.background = "green";
      capture.style.color = "white";

      stopCapture.textContent = "Stop Recording";
      stopCapture.style.background = "red";
      stopCapture.style.color = "white";
      div.appendChild(capture);

      const video = document.createElement('video');
      const recordingVid = document.createElement("video");
      video.style.display = 'block';

      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      // create a media recorder instance, which is an object
      // that will let you record what you stream.
      let recorder = new MediaRecorder(stream, options);
      document.body.appendChild(div);
      div.appendChild(video);
      // Video is a media element.  This line here sets the object which serves
      // as the source of the media associated with the HTMLMediaElement
      // Here, we'll set it equal to the stream.
      video.srcObject = stream;
      // We're inside an async function, so this await will fire off the playing
      // of a video. It returns a Promise which is resolved when playback has
      // been successfully started. Since this is async, the function will be
      // paused until this has started playing.
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      // and now, just wait for the capture button to get clicked in order to
      // start recording
      await new Promise((resolve) => {
        capture.onclick = resolve;
      });
      recorder.start();
      capture.replaceWith(stopCapture);
      // use a promise to tell it to stop recording
      await new Promise((resolve) => stopCapture.onclick = resolve);
      recorder.stop();

      let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
      let arrBuff = await recData.data.arrayBuffer();

      // stop the stream and remove the video element
      stream.getVideoTracks()[0].stop();
      div.remove();

      let binaryString = "";
      let bytes = new Uint8Array(arrBuff);
      bytes.forEach((byte) => {
        binaryString += String.fromCharCode(byte);
      })
      return btoa(binaryString);
    }
    """)
  try:
    display(js)
    data = eval_js('recordVideo({})')
    binary = b64decode(data)
    with open(filename, "wb") as video_file:
      video_file.write(binary)
    print(
        f"Finished recording video. Saved binary under filename in current working directory: {filename}"
    )
    '''
    unfortunately, webcam capture is not that clean in Colab and the videos need to
    be re-encoded to play properly. This takes some time.
    '''
    re_encode_video(filename)
  except Exception as err:
      # In case any exceptions arise
      print(str(err))
  return filename

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [31]:
# @title Constant
c_nc_cir_min = 0.85 # @param {type:"slider", min:0, max:1.01, step:0.01}
c_nc_midpt_tol = 1.0 # @param {type:"slider", min:0, max:2.01, step:0.01}
sample_area = 0.5 # @param {type:"slider", min:0, max:1.01, step:0.01}
area_filter = True # @param {type:"boolean"}

min_radius = 15 # @param {type:"integer"}
max_radius = 200 # @param {type:"integer"}

min_area = np.pi * (min_radius**2)
max_area = np.pi * (max_radius**2)

brightness = 10 # @param {type:"integer"}
contrast = 6 # @param {type:"integer"}
colour_mode = False # @param {type:"boolean"}

nc_label = "NC"
nc_color = (255, 0, 0)
c_label = "C"
c_color = (0, 255, 0)
pc_label = "PC"
pc_color = (0, 0, 225)

In [36]:
# @title Main Processing Code
def image_process(img : NDArray[np.floating], bbox_array : NDArray[np.floating]=None):
    # grayscale image
    if colour_mode:
        filteredImg = cv2.GaussianBlur(img, (7, 7), 0)
        filteredImg = cv2.cvtColor(filteredImg, cv2.COLOR_BGR2GRAY)
        filteredImg = cv2.convertScaleAbs(filteredImg, brightness, contrast)
        filteredImg = cv2.cvtColor(filteredImg, cv2.COLOR_GRAY2BGR)
    else:
        filteredImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Create Circle Detector
    params = cv2.SimpleBlobDetector_Params()

    # Filters
    params.minThreshold = 1

    params.filterByCircularity = True
    params.minCircularity = c_nc_cir_min

    params.filterByArea = area_filter
    params.minArea = min_area
    params.maxArea = max_area

    detector = cv2.SimpleBlobDetector_create(params)

    # Detect blobs in the image
    keypoints = detector.detect(filteredImg)
    colours = []
    sizes = []
    for keypoint in keypoints:
        total_colour = 0

        # Sampling
        for y_rg in range(-1, 2):
            for x_rg in range(-1, 2):
                cl = filteredImg[int(keypoint.pt[1] + y_rg*sample_area), int(keypoint.pt[0] + y_rg*sample_area)]
                if colour_mode:
                    cl = cl[0]**2
                else:
                    cl = cl**2
                total_colour += cl

        total_colour **= (0.5)
        colours.append(total_colour)

        sizes.append(keypoint.size)

    if len(colours) > 0:
        middle_c = (max(colours) + min(colours))/2
        middle_s = (max(sizes) + min(sizes))/2
        i = 0

        for keypoint in keypoints:
            x = int(keypoint.pt[0])
            y = int(keypoint.pt[1])
            r = int(keypoint.size / 2)

            is_c = sizes[i] >= middle_s * c_nc_midpt_tol if not colour_mode else colours[i] >= middle_c * c_nc_midpt_tol

            if is_c:
                if bbox_array is None:
                    img = cv2.circle(img, (x, y), r, c_color, 2)
                    img = cv2.putText(img, c_label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                else:
                    bbox_array = cv2.circle(bbox_array, (x, y), r, c_color, 2)
                    bbox_array = cv2.putText(bbox_array, c_label, (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            else:
                if bbox_array is None:
                    img = cv2.circle(img, (x, y), r, nc_color, 2)
                    img = cv2.putText(img, nc_label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                else:
                    bbox_array = cv2.circle(bbox_array, (x, y), r, nc_color, 2)
                    bbox_array = cv2.putText(bbox_array, nc_label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            i += 1

    # Detect Partial
    # Create Circle Detector
    params = cv2.SimpleBlobDetector_Params()

    # Filters
    params.minThreshold = 1

    params.filterByArea = area_filter
    params.minArea = min_area

    detector = cv2.SimpleBlobDetector_create(params)

    # Detect blobs in the image
    pc_keypoints = detector.detect(filteredImg)

    # Mark
    for pc_keypoint in pc_keypoints:
        x = int(pc_keypoint.pt[0])
        y = int(pc_keypoint.pt[1])
        r = int(pc_keypoint.size / 2)

        # Detect Pre-existing Detection
        alr_detected = False
        for kp in keypoints:
            x_k = int(kp.pt[0])
            y_k = int(kp.pt[1])
            r_k = int(kp.size / 2)

            # Detect Bounds
            if (x_k + r_k > x and x_k - r_k < x) and (y_k + r_k > y and y_k - y_k < y):
                alr_detected = True
                break

        if alr_detected:
            continue

        if bbox_array is None:
            img = cv2.putText(img, pc_label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            img = cv2.circle(img, (x, y), r, pc_color, 2)
        else:
            bbox_array = cv2.putText(bbox_array, pc_label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            bbox_array = cv2.circle(bbox_array, (x, y), r, pc_color, 2)

    return img, bbox_array, filteredImg

def start_live_processing():
    # start streaming video from webcam
    video_stream()
    # label for video
    label_html = 'Capturing...'
    # initialze bounding box to empty
    bbox = ''
    count = 0

    while True:
        js_reply = video_frame(label_html, bbox)
        if not js_reply:
            break

        # convert JS response to OpenCV Image
        img = js_to_image(js_reply["img"])

        # create transparent overlay for bounding box
        bbox_array = np.zeros([480,640,4], dtype=np.uint8)

        # Process Image
        (img, bbox_array, _) = image_process(img, bbox_array)

        bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
        # convert overlay of bbox into bytes
        bbox_bytes = bbox_to_bytes(bbox_array)
        # update bbox so next frame gets new overlay
        bbox = bbox_bytes

def static_processing(path_to_img, show_filtered_image=True):
    img = cv2.imread(path_to_img)

    (img, _, filtered_img) = image_process(img)

    plt.figure(figsize=(20,10))
    if show_filtered_image:
        plt.subplot(1, 2, 1)
        plt.imshow(filtered_img),plt.title(f'Blob Map of {path_to_img}',color='c')
        plt.subplot(1, 2, 2)
    plt.imshow(img),plt.title(f'Result {path_to_img}',color='c')
    plt.show()

In [41]:
# @title Live Streaming
start_live_processing()

<IPython.core.display.Javascript object>

In [40]:
# @title Static Image Procesing
show_filtered_image = True # @param {type:"boolean"}

files = os.listdir("./")
files = [f for f in files if os.path.isfile('./' + f) and f.endswith('.jpg')]
for i in files:
    static_processing(i, show_filtered_image)

Output hidden; open in https://colab.research.google.com to view.